# baseline

將`關鍵字`比對換成`向量相似度`比對。

In [1]:
pip install -q langgraph langchain langchain-community langchain-huggingface transformers bitsandbytes chromadb accelerate

In [3]:
!pip install -U bitsandbytes

> 請將目前使用關鍵字比對的 route_by_query，改為使用向量相似度進行分類，並設一個合理的相似度門檻，根據檢索結果的分數判斷是否走 RAG 流程。  
例如用向量相似度及自訂 threshold 決定要不要分到 retriever。

*斜體文字*> Hint：similarity_search_with_score(...)  
可參考去年的讀書會 R4：向量資料庫的基本操作

In [4]:
docs_text = """
火影代數	姓名	師傅	徒弟
初代	千手柱間	無明確記載	猿飛日斬、水戶門炎、轉寢小春
二代	千手扉間	千手柱間（兄長）	猿飛日斬、志村團藏、宇智波鏡等
三代	猿飛日斬	千手柱間、千手扉間	自來也、大蛇丸、千手綱手（傳說三忍）
四代	波風湊	自來也	旗木卡卡西、宇智波帶土、野原琳
五代	千手綱手	猿飛日斬	春野櫻、志乃等（主要為春野櫻）
六代	旗木卡卡西	波風湊	漩渦鳴人、宇智波佐助、春野櫻（第七班）
七代	漩渦鳴人	自來也、旗木卡卡西	木葉丸等（主要為木葉丸）
"""

In [5]:
# --- 模型載入與生成器定義 ---

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

model_id = "MediaTek-Research/Breeze-7B-Instruct-v1_0"

# 量化設定（使用 4-bit 載入）
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
)

# 載入 tokenizer 和模型
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=quant_config,
)

# HuggingFace 的生成管線（可用於 pipeline 生成）
generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.4,
    return_full_text=False  # 僅回傳模型生成的內容
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [7]:
import shutil

In [16]:
# 改寫重點：用向量相似度決定是否執行 RAG

from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# shutil.rmtree("document_store", ignore_errors=True)


embedding_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    encode_kwargs={"normalize_embeddings": True}
)

# 包裝成 Document 時加提示詞
docs = [Document(page_content=f"passage: {txt.strip()}") for txt in docs_text.strip().split("\n\n")]

persist_path = "document_store"
collection_name = "naruto_collection"
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    collection_name=collection_name
)

from typing_extensions import TypedDict, List

class RAGState(TypedDict):
    query: str
    docs: List[Document]
    answer: str
    score: float


# def retrieve_node(state: RAGState) -> RAGState:
#     query = state["query"]
#     results = vectorstore.similarity_search_with_relevance_scores(query, k=3)
#     docs = [doc for doc, score in results]
#     max_score = max([score for _, score in results], default=0.0)
#     return {"query": query, "docs": docs, "answer": "", "score": max_score}

def retrieve_node(state: RAGState) -> RAGState:
    query = state["query"]
    results = vectorstore.similarity_search_with_score(query, k=3)

    docs = [doc for doc, score in results]
    max_score = max([1 - score for _, score in results], default=0.0)

    return {
        "query": query,
        "docs": docs,
        "answer": "",
        "score": max_score  # 這裡就是相似度語意
    }

def route_by_similarity(state: RAGState) -> str:
    threshold = 0.50  # 可依 embedding 調整
    if state["score"] >= threshold:
        return "rag"
    else:
        return "direct"

def generate_node(state: RAGState) -> RAGState:
    query, docs = state["query"], state["docs"]
    context = "\n".join([d.page_content for d in docs])
    prompt = (
        f"你是一個知識型助手，請根據以下內容回答問題：\n\n"
        f"內容：{context}\n\n"
        f"問題：{query}\n\n回答："
    )
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {"query": query, "docs": docs, "answer": output, "score": state["score"]}


def direct_generate_node(state: RAGState) -> RAGState:
    query = state["query"]
    prompt = f"請回答以下問題：{query}\n\n回答："
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {"query": query, "docs": [], "answer": output, "score": state["score"]}


def route_by_similarity(state: RAGState) -> str:
    threshold = 0.50
    if state["score"] >= threshold:
        print(f"相似度 {state['score']:.2f} → 使用 RAG")
        return "rag"
    else:
        print(f"相似度 {state['score']:.2f} → 使用直接生成")
        return "direct"

from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

graph_builder = StateGraph(RAGState)

graph_builder.set_entry_point("retriever")
graph_builder.add_node("retriever", RunnableLambda(retrieve_node))
graph_builder.add_node("generator", RunnableLambda(generate_node))
graph_builder.add_node("direct_generator", RunnableLambda(direct_generate_node))

# 分流條件改為根據相似度分流
graph_builder.add_conditional_edges(
    source="retriever",
    path=RunnableLambda(route_by_similarity),
    path_map={
        "rag": "generator",
        "direct": "direct_generator",
    }
)

graph_builder.add_edge("generator", END)
graph_builder.add_edge("direct_generator", END)
graph = graph_builder.compile()

# 使用時初始 state 要多加 score 欄位：
# init_state = {"query": "火影是誰？", "docs": [], "answer": "", "score": 0.0}


In [13]:
print("開始對話吧（輸入 q 結束）")

while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    # 初始 State，score 記得給 0，retriever 會補上
    init_state: RAGState = {
        "query": user_input,
        "docs": [],
        "answer": "",
        "score": 0.0
    }

    # 呼叫 LangGraph
    result = graph.invoke(init_state)
    raw_output = result["answer"]

    # 處理輸出格式
    answer_text = raw_output.split("回答：")[-1].strip()
    print("回答：", answer_text)
    print("===" * 20, "\n")

開始對話吧（輸入 q 結束）
使用者: 誰是第四代火影?
相似度 0.68 → 使用 RAG
回答： 第四代火影是波風湊。

使用者: 第四代火影的師傅是誰?
相似度 0.70 → 使用 RAG
回答： 第四代火影的師傅是自來也。

使用者: 第四代火影的徒弟有哪些人?
相似度 0.73 → 使用 RAG
回答： 第四代火影的徒弟有旗木卡卡西、宇智波帶土、野原琳。

使用者: 相對論是誰發明的?
相似度 0.52 → 使用直接生成
回答： 相對論是由愛因斯坦（Albert Einstein）在 1905 年提出的。

使用者: q
掰啦！


In [ ]:
print("開始對話吧（輸入 q 結束）")

while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    init_state: RAGState = {
        "query": user_input,
        "docs": [],
        "answer": ""
    }

    result = graph.invoke(init_state)
    raw_output = result["answer"]

    answer_text = raw_output.split("回答：")[-1].strip()
    print("回答：", answer_text)
    print("===" * 20, "\n")

開始對話吧（輸入 q 結束）
使用者: 誰是第四代火影?
route: cosine_sim = 0.8227
跑到 → naruto
回答： 第四代火影是波風湊。

使用者: 第四代火影的師傅是誰?
route: cosine_sim = 0.8646
跑到 → naruto
回答： 第四代火影的師傅是自來也。

使用者: 第四代火影的徒弟有哪些人?
route: cosine_sim = 0.8726
跑到 → naruto
回答： 第四代火影波風湊的徒弟包括旗木卡卡西、宇智波帶土、野原琳。

使用者: 相對論是誰發明的?
route: cosine_sim = 0.1666
跑到 → general
回答： 相對論是由愛因斯坦（Albert Einstein）在 1905 年提出的。

使用者: q
掰啦！


# advance

改成能支援多輪問答（Multi-turn RAG），並能根據前面的query判斷問題。




> 請將 RAGState 加入 history 欄位，並在生成回答時，將歷史對話與當前問題一起組成 prompt。

> Hint：
```
class MultiTurnRAGState(TypedDict):  
    history: List[str]  
    query: str  
    docs: List[Document]  
    answer: str
```



In [15]:
docs_text = """
火影代數	姓名	師傅	徒弟
初代	千手柱間	無明確記載	猿飛日斬、水戶門炎、轉寢小春
二代	千手扉間	千手柱間（兄長）	猿飛日斬、志村團藏、宇智波鏡等
三代	猿飛日斬	千手柱間、千手扉間	自來也、大蛇丸、千手綱手（傳說三忍）
四代	波風湊	自來也	旗木卡卡西、宇智波帶土、野原琳
五代	千手綱手	猿飛日斬	春野櫻、志乃等（主要為春野櫻）
六代	旗木卡卡西	波風湊	漩渦鳴人、宇智波佐助、春野櫻（第七班）
七代	漩渦鳴人	自來也、旗木卡卡西	木葉丸等（主要為木葉丸）
"""

In [17]:
# ✅ 多輪問答版本：Multi-turn RAG 支援

from typing_extensions import TypedDict, List
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda

# ✅ 定義多輪狀態
class MultiTurnRAGState(TypedDict):
    history: List[str]
    query: str
    docs: List[Document]
    answer: str

# ✅ 更新 retriever node，根據上下文判斷查詢內容

def retrieve_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    # 可選：加入歷史上下文到查詢中
    full_query = "\n".join(state["history"][-2:] + [state["query"]])
    full_query = f"query: {full_query}"

    results = vectorstore.similarity_search_with_relevance_scores(full_query, k=3)
    docs = [doc for doc, _ in results]

    return {
        "history": state["history"],
        "query": state["query"],
        "docs": docs,
        "answer": ""
    }

# ✅ generator node，加入對應 context

def generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query, docs = state["query"], state["docs"]
    context = "\n".join([d.page_content for d in docs])

    prompt = (
        f"你是一位知識型助手，請根據以下內容回答問題：\n\n"
        f"內容：{context}\n\n"
        f"問題：{query}\n\n回答："
    )
    output = generator(prompt, max_new_tokens=300)[0]["generated_text"]

    return {
        "history": state["history"] + [query, output],
        "query": query,
        "docs": docs,
        "answer": output
    }

# ✅ direct answer node（不檢索）
def direct_generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query = state["query"]
    prompt = f"請回答以下問題：{query}\n\n回答："
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]

    return {
        "history": state["history"] + [query, output],
        "query": query,
        "docs": [],
        "answer": output
    }

# ✅ 條件分流：根據 query 內容決定是否走 RAG

def route_by_similarity(state: MultiTurnRAGState) -> str:
    keywords = ["火影", "歷代", "忍者"]
    if any(k in state["query"] for k in keywords):
        return "rag"
    else:
        return "direct"

# ✅ 建立 graph
rag_graph = StateGraph(MultiTurnRAGState)
rag_graph.set_entry_point("router")

rag_graph.add_node("router", RunnableLambda(lambda x: x))
rag_graph.add_node("retriever", RunnableLambda(retrieve_node))
rag_graph.add_node("generator", RunnableLambda(generate_node))
rag_graph.add_node("direct_generator", RunnableLambda(direct_generate_node))

rag_graph.add_conditional_edges(
    source="router",
    path=RunnableLambda(route_by_similarity),
    path_map={
        "rag": "retriever",
        "direct": "direct_generator"
    }
)

rag_graph.add_edge("retriever", "generator")
rag_graph.add_edge("generator", END)
rag_graph.add_edge("direct_generator", END)

rag_executor = rag_graph.compile()

# ✅ 啟動對話
print("開始對話吧（輸入 q 結束）")

state = {"history": [], "query": "", "docs": [], "answer": ""}

while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("結束對話")
        break

    state["query"] = user_input
    result = rag_executor.invoke(state)
    answer = result["answer"]
    print("AI 助理:", answer.strip())

    # 將回答更新到狀態中，準備下輪
    state = result


開始對話吧（輸入 q 結束）
使用者: 第四代火影是誰?
AI 助理: 第四代火影是波風湊。
使用者: 他的師父是誰?


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


AI 助理: 他的師父是張三豐。
使用者: q
結束對話


In [ ]:
global_history: List[str] = []

print("開始對話吧（輸入 q 結束）")
while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    state = {"history": global_history, "query": user_input}
    result = graph.invoke(state)

    answer = result["answer"].split("回答：")[-1].strip()
    print("AI 助理:", answer)
    print("===" * 60, "\n")

    global_history = result["history"]

開始對話吧（輸入 q 結束）
使用者: 第四代火影是誰?
route: cosine_sim = 0.8092
跑到 → retriever
retrieve combined query: '第四代火影是誰?'
AI 助理: 第四代火影是波風湊。

使用者: 他的師父是誰?
route: cosine_sim = 0.5529
跑到 → retriever
retrieve combined query: '第四代火影是誰?\n他的師父是誰?'
AI 助理: 第四代火影的師父是自來也。

使用者: 他的徒弟有哪些人?
route: cosine_sim = 0.6542
跑到 → retriever
retrieve combined query: '第四代火影是誰?\n他的師父是誰?\n他的徒弟有哪些人?'
AI 助理: 他的徒弟有以下人：旗木卡卡西、宇智波帶土、野原琳。

使用者: 相對論是他發明的嗎?
route: cosine_sim = 0.1118
跑到 → general
AI 助理: 相對論不是第四代火影所發明的。相對論是物理學家阿爾伯特·愛因斯坦在1905年提出的，他提出了廣義相對論，後人又提出狹義相對論。相對論主要是研究加速度和重力的關係，以及加速度和時間、空間的關係。

使用者: q
掰啦！
